# Abalone Rings Regression using Classical Machine Learning Algorithms

## Comparison on Linear Regression, Ridge Regression, Lasso Regression, Random Forest and Decision Tree

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [19]:
data = pd.read_csv('data/abalone.csv')

In [20]:
le = LabelEncoder()
data['Sex'] = le.fit_transform(data['Sex'])
data['Sex']

0       2
1       2
2       0
3       2
4       1
       ..
4172    0
4173    2
4174    2
4175    0
4176    2
Name: Sex, Length: 4177, dtype: int64

In [21]:
X = data.drop('Rings', axis=1)
y = data['Rings']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Decision Tree': DecisionTreeRegressor(random_state=42)
}

In [24]:
for name, model in models.items():
    print(f"{name}:")
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"\tMAE: {mae:.4f}")
    print(f"\tRMSE: {rmse:.4f}")
    print(f"\tR2 Score: {r2:.4f}")

Linear Regression:
	MAE: 1.6205
	RMSE: 2.2272
	R2 Score: 0.5418
Ridge Regression:
	MAE: 1.6296
	RMSE: 2.2356
	R2 Score: 0.5383
Lasso Regression:
	MAE: 2.3983
	RMSE: 3.2725
	R2 Score: 0.0107
Random Forest:
	MAE: 1.3563
	RMSE: 1.9506
	R2 Score: 0.6485
Decision Tree:
	MAE: 1.7225
	RMSE: 2.4651
	R2 Score: 0.4387
